# Line Tests

In [1]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import dpsimpy

# %matplotlib widget

epsilon = 1e-12

## SP PiLine with RLC elements

In [ ]:
time_step = 0.00005
final_time = 1
sim_name = "SP_PiLine_Elements"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n1 = dpsimpy.sp.SimNode("n1")
n2 = dpsimpy.sp.SimNode("n2")
vn1 = dpsimpy.sp.SimNode("vn1")
gnd = dpsimpy.sp.SimNode.gnd

resistance = 5
inductance = 0.16
capacitance = 1e-6
conductance = 1e-6

vs = dpsimpy.sp.ph1.VoltageSource("v_1")
vs.set_parameters(complex(100000, 0))
res = dpsimpy.sp.ph1.Resistor("R_line")
res.set_parameters(resistance)
ind = dpsimpy.sp.ph1.Inductor("L_line")
ind.set_parameters(inductance)
cap1 = dpsimpy.sp.ph1.Capacitor("Cp_1")
cap1.set_parameters(capacitance / 2)
cap2 = dpsimpy.sp.ph1.Capacitor("Cp_2")
cap2.set_parameters(capacitance / 2)
con1 = dpsimpy.sp.ph1.Resistor("Gp_1")
con1.set_parameters(2 / conductance)
con2 = dpsimpy.sp.ph1.Resistor("Gp_2")
con2.set_parameters(2 / conductance)

load = dpsimpy.sp.ph1.Resistor("R_load")
load.set_parameters(10000)

vs.connect([gnd, n1])
res.connect([n1, vn1])
ind.connect([vn1, n2])
cap1.connect([n1, gnd])
cap2.connect([n2, gnd])
con1.connect([n1, gnd])
con2.connect([n2, gnd])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(
    50, [n1, n2, vn1], [vs, res, ind, cap1, cap2, con1, con2, load]
)

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iline", "i_intf", ind)
logger.log_attribute("iload", "i_intf", load)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.SP)
sim.add_logger(logger)

sim.run()

In [ ]:
time_step = 0.00005
final_time = 1
sim_name = "SP_PiLine_Component"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n1 = dpsimpy.sp.SimNode("n1")
n2 = dpsimpy.sp.SimNode("n2")
gnd = dpsimpy.sp.SimNode.gnd

resistance = 5
inductance = 0.16
capacitance = 1e-6
conductance = 1e-6

vs = dpsimpy.sp.ph1.VoltageSource("v_1", dpsimpy.LogLevel.debug)
vs.set_parameters(complex(100000, 0))

line = dpsimpy.sp.ph1.PiLine("Line", dpsimpy.LogLevel.debug)
line.set_parameters(resistance, inductance, capacitance, conductance)

load = dpsimpy.sp.ph1.Resistor("R_load", dpsimpy.LogLevel.debug)
load.set_parameters(10000)

vs.connect([gnd, n1])
line.connect([n1, n2])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, line, load])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iline", "i_intf", line)
logger.log_attribute("iload", "i_intf", load)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.SP)
sim.add_logger(logger)

sim.run()

In [ ]:
work_dir = "logs/SP_PiLine_Elements/"
log_name = "SP_PiLine_Elements"
print(work_dir + log_name + ".csv")
piline_elements = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")
piline_elements_sp_shifted = ts.frequency_shift_list(piline_elements, 50)

In [ ]:
plt.figure()
plt.plot(
    piline_elements_sp_shifted["v1_shift"].time,
    piline_elements_sp_shifted["v1_shift"].values,
    label="v1_shift",
)
plt.plot(
    piline_elements_sp_shifted["v2_shift"].time,
    piline_elements_sp_shifted["v2_shift"].values,
    label="v2_shift",
    linestyle=":",
)
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(
    piline_elements_sp_shifted["iline_shift"].time,
    piline_elements_sp_shifted["iline_shift"].values,
    label="iline_shift",
)
plt.legend()
plt.show()

## SP PiLine composite model

In [ ]:
work_dir = "logs/SP_PiLine_Component/"
log_name = "SP_PiLine_Component"
print(work_dir + log_name + ".csv")
piline_component = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")
piline_component_sp_shifted = ts.frequency_shift_list(piline_component, 50)

In [ ]:
plt.figure()
plt.plot(
    piline_component_sp_shifted["v1_shift"].time,
    piline_component_sp_shifted["v1_shift"].values,
    label="v1_shift",
)
plt.plot(
    piline_component_sp_shifted["v2_shift"].time,
    piline_component_sp_shifted["v2_shift"].values,
    label="v2_shift",
    linestyle=":",
)
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(
    piline_component_sp_shifted["iline_shift"].time,
    piline_component_sp_shifted["iline_shift"].values,
    label="iline_shift",
)
plt.legend()
plt.show()

## Error for SP PiLine

In [ ]:
plt.figure()
for name in ["v1_shift", "v2_shift", "iline_shift"]:
    plt.plot(
        piline_elements_sp_shifted[name].time,
        piline_elements_sp_shifted[name].values
        - piline_component_sp_shifted[name].values,
        label=name + "_error",
    )
plt.legend()
plt.show()

## Assertion for SP PiLine

In [ ]:
errors_sp_shifted = []
for name in ["v1_shift", "v2_shift", "iline_shift"]:
    errors_sp_shifted.append(
        np.absolute(
            piline_elements_sp_shifted[name].values
            - piline_component_sp_shifted[name].values
        ).max()
    )
    print(name + ": " + str(errors_sp_shifted[-1]))
assert np.max(errors_sp_shifted) < epsilon

## DP PiLine with RLC elements

In [ ]:
time_step = 0.00005
final_time = 1
sim_name = "DP_PiLine_Elements"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n1 = dpsimpy.dp.SimNode("n1")
n2 = dpsimpy.dp.SimNode("n2")
vn1 = dpsimpy.dp.SimNode("vn1")
gnd = dpsimpy.dp.SimNode.gnd

resistance = 5
inductance = 0.16
capacitance = 1e-6
conductance = 1e-6

vs = dpsimpy.dp.ph1.VoltageSource("v_1")
vs.set_parameters(complex(100000, 0))
res = dpsimpy.dp.ph1.Resistor("R_line")
res.set_parameters(resistance)
ind = dpsimpy.dp.ph1.Inductor("L_line")
ind.set_parameters(inductance)
cap1 = dpsimpy.dp.ph1.Capacitor("Cp_1")
cap1.set_parameters(capacitance / 2)
cap2 = dpsimpy.dp.ph1.Capacitor("Cp_2")
cap2.set_parameters(capacitance / 2)
con1 = dpsimpy.dp.ph1.Resistor("Gp_1")
con1.set_parameters(2 / conductance)
con2 = dpsimpy.dp.ph1.Resistor("Gp_2")
con2.set_parameters(2 / conductance)

load = dpsimpy.dp.ph1.Resistor("R_load")
load.set_parameters(10000)

vs.connect([gnd, n1])
res.connect([n1, vn1])
ind.connect([vn1, n2])
cap1.connect([n1, gnd])
cap2.connect([n2, gnd])
con1.connect([n1, gnd])
con2.connect([n2, gnd])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(
    50, [n1, n2, vn1], [vs, res, ind, cap1, cap2, con1, con2, load]
)

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iline", "i_intf", ind)
logger.log_attribute("iload", "i_intf", load)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)

sim.run()

In [ ]:
time_step = 0.00005
final_time = 1
sim_name = "DP_PiLine_Component"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n1 = dpsimpy.dp.SimNode("n1")
n2 = dpsimpy.dp.SimNode("n2")
gnd = dpsimpy.dp.SimNode.gnd

resistance = 5
inductance = 0.16
capacitance = 1e-6
conductance = 1e-6

vs = dpsimpy.dp.ph1.VoltageSource("v_1")
vs.set_parameters(complex(100000, 0))

line = dpsimpy.dp.ph1.PiLine("Line")
line.set_parameters(resistance, inductance, capacitance, conductance)

load = dpsimpy.dp.ph1.Resistor("R_load")
load.set_parameters(10000)

vs.connect([gnd, n1])
line.connect([n1, n2])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, line, load])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iline", "i_intf", line)
logger.log_attribute("iload", "i_intf", load)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)

sim.run()

In [14]:
time_step = 0.00005
final_time = 1
sim_name = "DP_PiLine_Diakoptics"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n1 = dpsimpy.dp.SimNode("n1")
n2 = dpsimpy.dp.SimNode("n2")
gnd = dpsimpy.dp.SimNode.gnd

vs = dpsimpy.dp.ph1.VoltageSource("v_1")
vs.set_parameters(complex(100000, 0))

line = dpsimpy.dp.ph1.PiLine("Line")
line.set_parameters(5, 0.16, 1e-6)

load = dpsimpy.dp.ph1.Resistor("R_load")
load.set_parameters(10000)

vs.connect([gnd, n1])
line.connect([n1, n2])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, load])
sys.add_tear_component(line)

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_tearing_components(sys.tear_components)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)

sim.run()

In [ ]:
work_dir = "logs/DP_PiLine_Elements/"
log_name = "DP_PiLine_Elements"
print(work_dir + log_name + ".csv")
piline_elements = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")
piline_elements_dp_shifted = ts.frequency_shift_list(piline_elements, 50)

In [ ]:
plt.figure()
plt.plot(
    piline_elements_dp_shifted["v1_shift"].time,
    piline_elements_dp_shifted["v1_shift"].values,
    label="v1_shift",
)
plt.plot(
    piline_elements_dp_shifted["v2_shift"].time,
    piline_elements_dp_shifted["v2_shift"].values,
    label="v2_shift",
    linestyle=":",
)
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(
    piline_elements_dp_shifted["iline_shift"].time,
    piline_elements_dp_shifted["iline_shift"].values,
    label="iline_shift",
)
plt.legend()
plt.show()

## DP PiLine composite model

In [ ]:
work_dir = "logs/DP_PiLine_Component/"
log_name = "DP_PiLine_Component"
print(work_dir + log_name + ".csv")
piline_component = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")
piline_component_dp_shifted = ts.frequency_shift_list(piline_component, 50)

In [ ]:
plt.figure()
plt.plot(
    piline_component_dp_shifted["v1_shift"].time,
    piline_component_dp_shifted["v1_shift"].values,
    label="v1_shift",
)
plt.plot(
    piline_component_dp_shifted["v2_shift"].time,
    piline_component_dp_shifted["v2_shift"].values,
    label="v2_shift",
    linestyle=":",
)
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(
    piline_component_dp_shifted["iline_shift"].time,
    piline_component_dp_shifted["iline_shift"].values,
    label="iline_shift",
)
plt.legend()
plt.show()

## Error for DP PiLine

In [ ]:
plt.figure()
for name in ["v1_shift", "v2_shift", "iline_shift"]:
    plt.plot(
        piline_elements_dp_shifted[name].time,
        piline_elements_dp_shifted[name].values
        - piline_component_dp_shifted[name].values,
        label=name + "_error",
    )
plt.legend()
plt.show()

## Assertion for DP PiLine

In [ ]:
errors_dp_shifted = []
for name in ["v1_shift", "v2_shift", "iline_shift"]:
    errors_dp_shifted.append(
        np.absolute(
            piline_elements_dp_shifted[name].values
            - piline_component_dp_shifted[name].values
        ).max()
    )
    print(name + ": " + str(errors_dp_shifted[-1]))
assert np.max(errors_dp_shifted) < epsilon

## EMT PiLine with RLC elements

In [ ]:
time_step = 0.00005
final_time = 1
sim_name = "EMT_PiLine_Elements"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.ABC)
n2 = dpsimpy.emt.SimNode("n2", dpsimpy.PhaseType.ABC)
vn1 = dpsimpy.emt.SimNode("vn1", dpsimpy.PhaseType.ABC)
gnd = dpsimpy.emt.SimNode.gnd

resistance = 5
inductance = 0.16
capacitance = 1e-6
conductance = 1e-6

vs = dpsimpy.emt.ph3.VoltageSource("v_1", dpsimpy.LogLevel.debug)
vs.set_parameters(
    dpsimpy.Math.single_phase_variable_to_three_phase(complex(100000, 0)), 50
)
res = dpsimpy.emt.ph3.Resistor("R_line", dpsimpy.LogLevel.debug)
res.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(resistance))
ind = dpsimpy.emt.ph3.Inductor("L_line", dpsimpy.LogLevel.debug)
ind.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(inductance))
cap1 = dpsimpy.emt.ph3.Capacitor("Cp_1", dpsimpy.LogLevel.debug)
cap1.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(capacitance / 2))
cap2 = dpsimpy.emt.ph3.Capacitor("Cp_2", dpsimpy.LogLevel.debug)
cap2.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(capacitance / 2))
con1 = dpsimpy.emt.ph3.Resistor("Gp_1", dpsimpy.LogLevel.debug)
con1.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(2 / conductance))
con2 = dpsimpy.emt.ph3.Resistor("Gp_2", dpsimpy.LogLevel.debug)
con2.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(2 / conductance))

load = dpsimpy.emt.ph3.Resistor("R_load", dpsimpy.LogLevel.debug)
load.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(10000))

vs.connect([gnd, n1])
res.connect([n1, vn1])
ind.connect([vn1, n2])
cap1.connect([n1, gnd])
cap2.connect([n2, gnd])
con1.connect([n1, gnd])
con2.connect([n2, gnd])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(
    50, [n1, n2, vn1], [vs, res, ind, cap1, cap2, con1, con2, load]
)

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iline", "i_intf", ind)

sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim.set_system(sys)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)

sim.run()

In [ ]:
time_step = 0.00005
final_time = 1
sim_name = "EMT_PiLine_Component"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.ABC)
n2 = dpsimpy.emt.SimNode("n2", dpsimpy.PhaseType.ABC)
gnd = dpsimpy.emt.SimNode.gnd

resistance = 5
inductance = 0.16
capacitance = 1e-6
conductance = 1e-6

vs = dpsimpy.emt.ph3.VoltageSource("v_1", dpsimpy.LogLevel.debug)
vs.set_parameters(
    dpsimpy.Math.single_phase_variable_to_three_phase(complex(100000, 0)), 50
)

line = dpsimpy.emt.ph3.PiLine("Line", dpsimpy.LogLevel.debug)
line.set_parameters(
    dpsimpy.Math.single_phase_parameter_to_three_phase(resistance),
    dpsimpy.Math.single_phase_parameter_to_three_phase(inductance),
    dpsimpy.Math.single_phase_parameter_to_three_phase(capacitance),
    dpsimpy.Math.single_phase_parameter_to_three_phase(conductance),
)

load = dpsimpy.emt.ph3.Resistor("R_load", dpsimpy.LogLevel.debug)
load.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(10000))

vs.connect([gnd, n1])
line.connect([n1, n2])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, line, load])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iline", "i_intf", line)
logger.log_attribute("iload", "i_intf", load)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)

sim.run()

In [ ]:
work_dir = "logs/EMT_PiLine_Elements/"
log_name = "EMT_PiLine_Elements"
print(work_dir + log_name + ".csv")
piline_elements_emt = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")

In [ ]:
plt.figure()
plt.plot(
    piline_elements_emt["v1_0"].time, piline_elements_emt["v1_0"].values, label="v1_0"
)
plt.plot(
    piline_elements_emt["v2_0"].time,
    piline_elements_emt["v2_0"].values,
    label="v2_0",
    linestyle=":",
)
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(
    piline_elements_emt["iline_0"].time,
    piline_elements_emt["iline_0"].values,
    label="iline_0",
)
plt.legend()
plt.show()

## EMT PiLine composite model

In [ ]:
work_dir = "logs/EMT_PiLine_Component/"
log_name = "EMT_PiLine_Component"
print(work_dir + log_name + ".csv")
piline_component_emt = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")

In [ ]:
plt.figure()
plt.plot(
    piline_component_emt["v1_0"].time, piline_component_emt["v1_0"].values, label="v1_0"
)
plt.plot(
    piline_component_emt["v2_0"].time,
    piline_component_emt["v2_0"].values,
    label="v2_0",
    linestyle=":",
)
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(
    piline_component_emt["iline_0"].time,
    piline_component_emt["iline_0"].values,
    label="iline_0",
)
plt.legend()
plt.show()

## Error for EMT PiLine

In [ ]:
plt.figure()
for name in ["v1_0", "v2_0", "iline_0"]:
    plt.plot(
        piline_elements_emt[name].time,
        piline_elements_emt[name].values - piline_component_emt[name].values,
        label=name + "_error",
    )
plt.legend()
plt.show()

## Assertion for EMT PiLine

In [ ]:
errors_emt = []
for name in ["v1_0", "v2_0", "iline_0"]:
    errors_emt.append(
        np.absolute(
            piline_elements_emt[name].values - piline_component_emt[name].values
        ).max()
    )
    print(name + ": " + str(errors_emt[-1]))
assert np.max(errors_emt) < epsilon

## Compare PiLine Component DP vs. EMT

In [ ]:
plt.figure()
plt.plot(
    piline_component_emt["v1_0"].time,
    np.sqrt(3 / 2) * piline_component_emt["v1_0"].values
    - piline_component_dp_shifted["v1_shift"].values,
    label="v1_0 vs. v1_shift",
)
plt.plot(
    piline_component_emt["v2_0"].time,
    np.sqrt(3 / 2) * piline_component_emt["v2_0"].values
    - piline_component_dp_shifted["v2_shift"].values,
    label="v2_0 vs. v2_shift",
    linestyle=":",
)
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(
    piline_component_emt["iline_0"].time,
    np.sqrt(3 / 2) * piline_component_emt["iline_0"].values
    - piline_component_dp_shifted["iline_shift"].values,
    label="iline_0" + " vs. " + "iline_shift",
)
plt.legend()
plt.show()

## Assertion DP vs. EMT for PiLine Component

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [("v1_0", "v1_shift"), ("v2_0", "v2_shift"), ("iline_0", "iline_shift")]:
    compare_errors_abs.append(
        np.absolute(
            np.sqrt(3 / 2) * piline_component_emt[name[0]].values
            - piline_component_dp_shifted[name[1]].values
        ).max()
    )
    compare_errors_rel.append(
        np.absolute(
            np.sqrt(3 / 2) * piline_component_emt[name[0]].values
            - piline_component_dp_shifted[name[1]].values
        ).max()
        / piline_component_dp_shifted[name[1]].values.max()
    )
    print(name[0] + " vs. " + name[1] + " (abs): " + str(compare_errors_abs[-1]))
    print(name[0] + " vs. " + name[1] + " (rel): " + str(compare_errors_rel[-1]))
print("Max rel error: " + "{:.2}".format(np.max(compare_errors_rel) * 100) + "%")
assert np.max(compare_errors_rel) < 1e-3

## Diakoptics PI-Line

In [ ]:
work_dir = "logs/DP_PiLine_Diakoptics/"
log_name = "DP_PiLine_Diakoptics"
print(work_dir + log_name + ".csv")
piline_diakoptics = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")
piline_diakoptics_emt = ts.frequency_shift_list(piline_diakoptics, 50)

In [ ]:
plt.figure()
plt.plot(
    piline_diakoptics_emt["v1_shift"].time, piline_diakoptics_emt["v1_shift"].values
)
plt.plot(
    piline_diakoptics_emt["v2_shift"].time, piline_diakoptics_emt["v2_shift"].values
)
# plt.xlim([0,0.02])
plt.show()

## Decoupling Line

In [ ]:
time_step = 0.00005
final_time = 0.1
sim_name = "DP_Decoupling_Elements"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n1 = dpsimpy.dp.SimNode("n1")
n2 = dpsimpy.dp.SimNode("n2")
vn1 = dpsimpy.dp.SimNode("vn1")
gnd = dpsimpy.dp.SimNode.gnd

resistance = 5
inductance = 0.16
capacitance = 1e-6

vs = dpsimpy.dp.ph1.VoltageSource("v_1")
vs.set_parameters(100000 * np.exp(-np.pi * 1j / 2))
res = dpsimpy.dp.ph1.Resistor("R_line")
res.set_parameters(resistance)
ind = dpsimpy.dp.ph1.Inductor("L_line")
ind.set_parameters(inductance)
cap1 = dpsimpy.dp.ph1.Capacitor("Cp_1")
cap1.set_parameters(capacitance / 2)
cap2 = dpsimpy.dp.ph1.Capacitor("Cp_2")
cap2.set_parameters(capacitance / 2)

load = dpsimpy.dp.ph1.Resistor("R_load")
load.set_parameters(10000)

vs.connect([gnd, n1])
res.connect([n1, vn1])
ind.connect([vn1, n2])
cap1.connect([n1, gnd])
cap2.connect([n2, gnd])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2, vn1], [vs, res, ind, cap1, cap2, load])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)

sim.run()

In [ ]:
time_step = 0.00005
final_time = 0.1
sim_name = "DP_Decoupling_Wave"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n1 = dpsimpy.dp.SimNode("n1")
n2 = dpsimpy.dp.SimNode("n2")
gnd = dpsimpy.dp.SimNode.gnd

resistance = 5
inductance = 0.16
capacitance = 1e-6

vs = dpsimpy.dp.ph1.VoltageSource("Vsrc")
vs.set_parameters(complex(100000, 0))

dline = dpsimpy.signal.DecouplingLine("DecLine", dpsimpy.LogLevel.debug)
dline.set_parameters(n1, n2, resistance, inductance, capacitance)

load = dpsimpy.dp.ph1.Resistor("R_load")
load.set_parameters(10000)

vs.connect([gnd, n1])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, dline, load])
sys.add(dline.get_line_components())

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("i1", "i_intf", vs)
logger.log_attribute("i2", "i_intf", load)
logger.log_attribute("i_src1", "i_src1", dline)
logger.log_attribute("i_src2", "i_src2", dline)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)

sim.run()

In [ ]:
time_step = 0.00005
final_time = 0.1
sim_name = "EMT_Decoupling_Wave"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n1 = dpsimpy.emt.SimNode("n1")
n2 = dpsimpy.emt.SimNode("n2")
gnd = dpsimpy.emt.SimNode.gnd

resistance = 5
inductance = 0.16
capacitance = 1e-6

vs = dpsimpy.emt.ph1.VoltageSource("Vsrc_emt")
vs.set_parameters(complex(100000, 0), 50)

dline = dpsimpy.signal.DecouplingLineEMT("DecLine_emt", dpsimpy.LogLevel.debug)
dline.set_parameters(n1, n2, resistance, inductance, capacitance)

load = dpsimpy.emt.ph1.Resistor("R_load")
load.set_parameters(10000)

vs.connect([gnd, n1])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, dline, load])
sys.add(dline.get_line_components())

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("i1", "i_intf", vs)
logger.log_attribute("i2", "i_intf", load)
logger.log_attribute("i_src1", "i_src1", dline)
logger.log_attribute("i_src2", "i_src2", dline)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)

sim.run()

In [ ]:
time_step = 0.00005
final_time = 0.1
sim_name = "EMT_Decoupling_Wave_Ph3"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.ABC)
n2 = dpsimpy.emt.SimNode("n2", dpsimpy.PhaseType.ABC)
gnd = dpsimpy.emt.SimNode.gnd

resistance = dpsimpy.Math.single_phase_parameter_to_three_phase(5)
inductance = dpsimpy.Math.single_phase_parameter_to_three_phase(0.16)
capacitance = dpsimpy.Math.single_phase_parameter_to_three_phase(1e-6)

vs = dpsimpy.emt.ph3.VoltageSource("Vsrc_emt")
vs_ref_rms = 100000 * dpsimpy.PEAK1PH_TO_RMS3PH
vs_ref_ph3 = dpsimpy.Math.single_phase_variable_to_three_phase(complex(vs_ref_rms, 0))
vs.set_parameters(vs_ref_ph3, 50)

dline = dpsimpy.signal.DecouplingLineEMT_Ph3("DecLine_emt_ph3", dpsimpy.LogLevel.debug)
dline.set_parameters(n1, n2, resistance, inductance, capacitance)

load = dpsimpy.emt.ph3.Resistor("R_load")
load.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(10000))

vs.connect([gnd, n1])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2], [vs, dline, load])
sys.add(dline.get_line_components())

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("i1", "i_intf", vs)
logger.log_attribute("i2", "i_intf", load)
logger.log_attribute("i_src1", "i_src1", dline)
logger.log_attribute("i_src2", "i_src2", dline)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)

sim.run()

In [ ]:
work_dir = "logs/DP_Decoupling_Wave/"
log_name = "DP_Decoupling_Wave"
print(work_dir + log_name + ".csv")
decoupling_wave = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")
decoupling_wave_emt = ts.frequency_shift_list(decoupling_wave, 50)

In [ ]:
plt.figure()
plt.plot(decoupling_wave_emt["v1_shift"].time, decoupling_wave_emt["v1_shift"].values)
plt.plot(decoupling_wave_emt["v2_shift"].time, decoupling_wave_emt["v2_shift"].values)
# plt.xlim([0,0.02])
plt.show()

In [ ]:
plt.figure()
plt.plot(decoupling_wave_emt["i1_shift"].time, decoupling_wave_emt["i1_shift"].values)
plt.plot(decoupling_wave_emt["i2_shift"].time, decoupling_wave_emt["i2_shift"].values)
# plt.xlim([0.02,0.1])
# plt.ylim([-20,20])
plt.show()

In [ ]:
plt.figure()
plt.plot(
    decoupling_wave_emt["i_src1_shift"].time, decoupling_wave_emt["i_src1_shift"].values
)
plt.plot(
    decoupling_wave_emt["i_src2_shift"].time, decoupling_wave_emt["i_src2_shift"].values
)
# plt.xlim([0.02,0.1])
# plt.ylim([-20,20])
plt.show()

In [ ]:
i_src1_phasor = decoupling_wave["i_src1"].phasor()
i_src2_phasor = decoupling_wave["i_src2"].phasor()
plt.figure()
plt.plot(i_src1_phasor["abs"].time, i_src1_phasor["abs"].values)
plt.plot(i_src2_phasor["abs"].time, i_src2_phasor["abs"].values)
# plt.xlim([0.04,0.06])
# plt.ylim([220,280])
plt.show()

In [ ]:
work_dir = "logs/EMT_Decoupling_Wave/"
log_name = "EMT_Decoupling_Wave"
print(work_dir + log_name + ".csv")
emt_decoupling_wave = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")

In [ ]:
plt.figure()
plt.plot(emt_decoupling_wave["v1"].time, emt_decoupling_wave["v1"].values)
plt.plot(emt_decoupling_wave["v2"].time, emt_decoupling_wave["v2"].values)
# plt.xlim([0,0.02])
plt.show()

In [ ]:
plt.figure()
plt.plot(emt_decoupling_wave["i_src1"].time, emt_decoupling_wave["i_src1"].values)
plt.plot(emt_decoupling_wave["i_src2"].time, emt_decoupling_wave["i_src2"].values)
# plt.xlim([0,0.02])
plt.show()

In [ ]:
plt.figure()
plt.plot(emt_decoupling_wave["i1"].time, emt_decoupling_wave["i1"].values)
plt.plot(emt_decoupling_wave["i2"].time, emt_decoupling_wave["i2"].values)
# plt.xlim([0,0.02])
plt.show()

In [ ]:
work_dir = "logs/EMT_Decoupling_Wave_Ph3/"
log_name = "EMT_Decoupling_Wave_Ph3"
print(work_dir + log_name + ".csv")
emt_decoupling_wave_ph3 = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")

In [ ]:
plt.figure()
plt.plot(emt_decoupling_wave_ph3["v1_0"].time, emt_decoupling_wave_ph3["v1_0"].values)
plt.plot(emt_decoupling_wave_ph3["v2_0"].time, emt_decoupling_wave_ph3["v2_0"].values)
# plt.xlim([0,0.02])
plt.show()

In [ ]:
plt.figure()
plt.plot(
    emt_decoupling_wave_ph3["i_src1_0"].time, emt_decoupling_wave_ph3["i_src1_0"].values
)
plt.plot(
    emt_decoupling_wave_ph3["i_src2_0"].time, emt_decoupling_wave_ph3["i_src2_0"].values
)
# plt.xlim([0,0.02])
plt.show()

In [ ]:
plt.figure()
plt.plot(emt_decoupling_wave_ph3["i1_0"].time, emt_decoupling_wave_ph3["i1_0"].values)
plt.plot(emt_decoupling_wave_ph3["i2_0"].time, emt_decoupling_wave_ph3["i2_0"].values)
# plt.xlim([0,0.02])
plt.show()

In [ ]:
plt.figure()
plt.plot(
    emt_decoupling_wave["i1"].time,
    emt_decoupling_wave["i1"].values,
    color="#939393ff",
    label="i1_emt",
)
plt.plot(
    emt_decoupling_wave["i2"].time,
    emt_decoupling_wave["i2"].values,
    color="#939393ff",
    label="i2_emt",
)
plt.plot(
    decoupling_wave_emt["i1_shift"].time,
    decoupling_wave_emt["i1_shift"].values,
    linestyle=":",
    color="#0055ffff",
    label="i1_dp",
)
plt.plot(
    decoupling_wave_emt["i2_shift"].time,
    decoupling_wave_emt["i2_shift"].values,
    linestyle=":",
    color="#d62728ff",
    label="i2_dp",
)
plt.legend()
plt.xlabel("time (s)")
plt.ylabel("current (A)")
# plt.xlim([0.02,0.1])
# plt.ylim([-20,20])
plt.savefig("tlm_dp_emt_current.pdf")

In [ ]:
plt.figure()
plt.plot(
    emt_decoupling_wave["v1"].time,
    emt_decoupling_wave["v1"].values,
    color="#939393ff",
    label="v1_emt",
)
plt.plot(
    emt_decoupling_wave["v2"].time,
    emt_decoupling_wave["v2"].values,
    color="#939393ff",
    label="v2_emt",
)
plt.plot(
    decoupling_wave_emt["v1_shift"].time,
    decoupling_wave_emt["v1_shift"].values,
    linestyle=":",
    color="#0055ffff",
    label="v1_dp",
)
plt.plot(
    decoupling_wave_emt["v2_shift"].time,
    decoupling_wave_emt["v2_shift"].values,
    linestyle=":",
    color="#d62728ff",
    label="v2_dp",
)
plt.legend()
plt.xlabel("time (s)")
plt.ylabel("voltage (V)")
# plt.xlim([0.02,0.1])
# plt.ylim([-20,20])
plt.savefig("tlm_dp_emt_voltage.pdf")

In [ ]:
plt.figure()
plt.plot(
    emt_decoupling_wave["v1"].time,
    emt_decoupling_wave["v1"].values,
    color="#939393ff",
    label="v1_emt",
)
plt.plot(
    emt_decoupling_wave["v2"].time,
    emt_decoupling_wave["v2"].values,
    color="#939393ff",
    label="v2_emt",
)
plt.plot(
    emt_decoupling_wave_ph3["v1_0"].time,
    emt_decoupling_wave_ph3["v1_0"].values,
    linestyle=":",
    color="#0055ffff",
    label="v1_emt_ph3",
)
plt.plot(
    emt_decoupling_wave_ph3["v2_0"].time,
    emt_decoupling_wave_ph3["v2_0"].values,
    linestyle=":",
    color="#d62728ff",
    label="v2_emt_ph3",
)
plt.legend()
plt.xlabel("time (s)")
plt.ylabel("voltage (V)")
# plt.xlim([0.02,0.1])
# plt.ylim([-20,20])
plt.savefig("tlm_emt_emt3ph_voltage.pdf")

In [ ]:
plt.figure()
plt.plot(
    emt_decoupling_wave["i_src1"].time,
    emt_decoupling_wave["i_src1"].values,
    color="#939393ff",
    label="i_src1_emt",
)
plt.plot(
    emt_decoupling_wave["i_src2"].time,
    emt_decoupling_wave["i_src2"].values,
    color="#939393ff",
    label="i_src2_emt",
)
plt.plot(
    emt_decoupling_wave_ph3["i_src1_0"].time,
    emt_decoupling_wave_ph3["i_src1_0"].values,
    linestyle=":",
    color="#0055ffff",
    label="i_src1_emt_ph3",
)
plt.plot(
    emt_decoupling_wave_ph3["i_src2_0"].time,
    emt_decoupling_wave_ph3["i_src2_0"].values,
    linestyle=":",
    color="#d62728ff",
    label="i_src2_emt_ph3",
)
plt.legend()
plt.xlabel("time (s)")
plt.ylabel("current (A)")
# plt.xlim([0.02,0.1])
# plt.ylim([-20,20])
plt.savefig("tlm_emt_emt3ph_current.pdf")